In [51]:
options = ['piedra', 'papel','tijeras']

In [52]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    elif p1 == 'piedra' and p2 == 'tijeras':
        result = 1
    elif p1 == 'piedra' and p2 == 'papel':
        result = 2
    elif p1 == 'tijeras' and p2 == 'piedra':
        result = 2
    elif p1 == 'tijeras' and p2 == 'papel':
        result = 1
    elif p1 == 'papel' and p2 == 'piedra':
        result = 1
    elif p1 == 'papel' and p2 == 'tijeras':
        result = 2
    return result
    

In [53]:
test = [
    ['piedra','piedra',0],
    ['piedra','tijeras',1],
    ['piedra','papel',2]
]

for partida in test:
    print("Player 1: %s Player 2: %s Winner: %s Validation: %s" % 
          (partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]))

Player 1: piedra Player 2: piedra Winner: 0 Validation: 0
Player 1: piedra Player 2: tijeras Winner: 1 Validation: 1
Player 1: piedra Player 2: papel Winner: 2 Validation: 2


In [54]:
from random import choice
def get_choice():
    return choice(options)

In [55]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("Player 1: %s Player 2: %s Winner: %s " % 
          (player1, player2, search_winner(player1, player2)
          ))
    

Player 1: piedra Player 2: piedra Winner: 0 
Player 1: papel Player 2: papel Winner: 0 
Player 1: tijeras Player 2: papel Winner: 1 
Player 1: piedra Player 2: papel Winner: 2 
Player 1: piedra Player 2: papel Winner: 2 
Player 1: papel Player 2: piedra Winner: 1 
Player 1: papel Player 2: papel Winner: 0 
Player 1: papel Player 2: tijeras Winner: 2 
Player 1: piedra Player 2: piedra Winner: 0 
Player 1: tijeras Player 2: tijeras Winner: 0 


In [56]:
def str_to_list(option):
    if option == "piedra":
        res = [1,0,0]
    elif option == "tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ['piedra','tijeras','papel']))
data_Y = list(map(str_to_list, ['papel','piedra','tijeras']))

print(data_X)
print(data_Y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [57]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(verbose=False,warm_start=True)

In [58]:
model = clf.fit([data_X[0]],[data_Y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=True)


/Users/marcosdonoso/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [59]:
def play_and_learn(iters=10, debug=False):
    score = {"win":0,"loose":0}

    data_X = []
    data_Y = []

    for i in range(iters):
        player1 = get_choice()

        predict = model.predict_proba([str_to_list(player1)])[0]

        if predict[0] >= 0.94:
            player2 = options[0]
        elif predict[1] >= 0.94:
            player2 = options[1]
        elif predict[2] >= 0.94:
            player2 = options[2]
        else:
            player2 = get_choice()

        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))

        winner = search_winner(player1,player2)
        if debug==True:
            print("Comprobamos Player1 VS  Player2  %s " % winner)

        if winner==2:
            data_X.append(str_to_list(player1))
            data_Y.append(str_to_list(player2))

            score["win"]+=1
        else:
            score["loose"]+=1
    return score, data_X, data_Y

In [60]:
score, data_X, data_Y = play_and_learn(1, debug=True)
print(data_X)
print(data_Y)
print("Score: %s %s %%" % (score, (score["win"]%100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_Y)    

Player1: piedra Player2 (modelo): [0.03898313 0.05117892 0.95124962] --> tijeras
Comprobamos Player1 VS  Player2  1 
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [61]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_Y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))

    if len(data_X):
        model = model.partial_fit(data_X, data_Y)

    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 226, 'loose': 774} 22.6 %
Iter: 2 - score: {'win': 240, 'loose': 760} 24.0 %
Iter: 3 - score: {'win': 214, 'loose': 786} 21.4 %
Iter: 4 - score: {'win': 223, 'loose': 777} 22.3 %
Iter: 5 - score: {'win': 215, 'loose': 785} 21.5 %
Iter: 6 - score: {'win': 209, 'loose': 791} 20.9 %
Iter: 7 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 8 - score: {'win': 350, 'loose': 650} 35.0 %
Iter: 9 - score: {'win': 315, 'loose': 685} 31.5 %
Iter: 10 - score: {'win': 344, 'loose': 656} 34.4 %
Iter: 11 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 12 - score: {'win': 311, 'loose': 689} 31.1 %
Iter: 13 - score: {'win': 359, 'loose': 641} 35.9 %
Iter: 14 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 15 - score: {'win': 348, 'loose': 652} 34.8 %
Iter: 16 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 17 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 18 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 19 - score: {'win': 352, 'loose': 648} 35.2 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 537, 'loose': 463} 53.7 %
Iter: 159 - score: {'win': 580, 'loose': 420} 58.0 %
Iter: 160 - score: {'win': 572, 'loose': 428} 57.2 %
Iter: 161 - score: {'win': 579, 'loose': 421} 57.9 %
Iter: 162 - score: {'win': 419, 'loose': 581} 41.9 %
Iter: 163 - score: {'win': 439, 'loose': 561} 43.9 %
Iter: 164 - score: {'win': 557, 'loose': 443} 55.7 %
Iter: 165 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 166 - score: {'win': 545, 'loose': 455} 54.5 %
Iter: 167 - score: {'win': 577, 'loose': 423} 57.7 %
Iter: 168 - score: {'win': 439, 'loose': 561} 43.9 %
Iter: 169 - score: {'win': 454, 'loose': 546} 45.4 %
Iter: 170 - score: {'win': 565, 'loose': 435} 56.5 %
Iter: 171 - score: {'win': 510, 'loose': 490} 51.0 %
Iter: 172 - score: {'win': 537, 'loose': 463} 53.7 %
Iter: 173 - score: {'win': 461, 'loose': 539} 46.1 %
Iter: 174 - score: {'win': 446, 'loose': 554} 44.6 %
Iter: 175 - score: {'win': 437, 'loose': 563} 43.7 %
Iter: 176 - score: {'win': 580, 'loose': 420} 

Iter: 313 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 314 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 315 - score: {'win': 437, 'loose': 563} 43.7 %
Iter: 316 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 317 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 318 - score: {'win': 320, 'loose': 680} 32.0 %
Iter: 319 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 320 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 321 - score: {'win': 309, 'loose': 691} 30.9 %
Iter: 322 - score: {'win': 350, 'loose': 650} 35.0 %
Iter: 323 - score: {'win': 460, 'loose': 540} 46.0 %
Iter: 324 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 325 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 326 - score: {'win': 318, 'loose': 682} 31.8 %
Iter: 327 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 328 - score: {'win': 429, 'loose': 571} 42.9 %
Iter: 329 - score: {'win': 314, 'loose': 686} 31.4 %
Iter: 330 - score: {'win': 349, 'loose': 651} 34.9 %
Iter: 331 - score: {'win': 321, 'loose': 679} 

Iter: 469 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 470 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 471 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 472 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 473 - score: {'win': 437, 'loose': 563} 43.7 %
Iter: 474 - score: {'win': 297, 'loose': 703} 29.7 %
Iter: 475 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 476 - score: {'win': 432, 'loose': 568} 43.2 %
Iter: 477 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 478 - score: {'win': 304, 'loose': 696} 30.4 %
Iter: 479 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 480 - score: {'win': 315, 'loose': 685} 31.5 %
Iter: 481 - score: {'win': 352, 'loose': 648} 35.2 %
Iter: 482 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 483 - score: {'win': 321, 'loose': 679} 32.1 %
Iter: 484 - score: {'win': 306, 'loose': 694} 30.6 %
Iter: 485 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 486 - score: {'win': 318, 'loose': 682} 31.8 %
Iter: 487 - score: {'win': 356, 'loose': 644} 

Iter: 624 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 625 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 626 - score: {'win': 357, 'loose': 643} 35.7 %
Iter: 627 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 628 - score: {'win': 297, 'loose': 703} 29.7 %
Iter: 629 - score: {'win': 586, 'loose': 414} 58.6 %
Iter: 630 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 631 - score: {'win': 308, 'loose': 692} 30.8 %
Iter: 632 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 633 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 634 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 635 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 636 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 637 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 638 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 639 - score: {'win': 321, 'loose': 679} 32.1 %
Iter: 640 - score: {'win': 354, 'loose': 646} 35.4 %
Iter: 641 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 642 - score: {'win': 320, 'loose': 680} 

Iter: 780 - score: {'win': 314, 'loose': 686} 31.4 %
Iter: 781 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 782 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 783 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 784 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 785 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 786 - score: {'win': 320, 'loose': 680} 32.0 %
Iter: 787 - score: {'win': 328, 'loose': 672} 32.8 %
Iter: 788 - score: {'win': 345, 'loose': 655} 34.5 %
Iter: 789 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 790 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 791 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 792 - score: {'win': 446, 'loose': 554} 44.6 %
Iter: 793 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 794 - score: {'win': 336, 'loose': 664} 33.6 %
Iter: 795 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 796 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 797 - score: {'win': 325, 'loose': 675} 32.5 %
Iter: 798 - score: {'win': 359, 'loose': 641} 

KeyboardInterrupt: 